In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import time
import ipywidgets as wg
from ipywidgets import interact
from IPython.display import display
import networkx as nx
from src.environment import EnvironmentNetwork
from src.runner import Runner
import random
import json

# Experiments

There are three initial experiments which are compared to a baseline of no policy interventions. These three scenarios are:

1. A five week lock-down
2. A five week lock-down but without effect in the Townships
3. A lock down for vulnerable people

### Experiment 1: five week lockdown

### Experiment 2: five week lockdown but less effect in informal settlements. 

In [4]:
informal_residential = pd.read_csv('Informal_Residential.csv').iloc[:-1]

In [5]:
population = pd.read_csv('population.csv')

In [9]:
population['Informal_residential'] = informal_residential['Informal_residential']

In [10]:
population

,WardID,Population,Area,Density,lon,lat,Informal_residential
0,19100001,26434.07345,27213.359916,0.971364,18.612025,-33.857307,0.000000
1,19100002,29914.18993,9222.458013,3.243624,18.422324,-34.069837,0.000000
2,19100003,25735.62250,10891.277732,2.362957,18.476290,-34.048938,0.000000
3,19100004,33364.15882,12636.140262,2.640376,18.461126,-34.033715,0.000000
4,19100005,24480.76812,8859.077165,2.763354,18.355432,-34.027518,0.000000
5,19100006,36512.04739,8092.255413,4.511974,18.591930,-34.030908,0.146332
6,19100007,27867.71894,8922.720819,3.123231,18.613178,-34.027953,0.000000
7,19100008,29266.33749,16516.898306,1.771903,18.414747,-33.947378,0.000000
8,19100009,31735.90931,11129.106450,2.851613,18.600669,-34.048995,0.000000
9,19100010,30520.92345,11059.814862,2.759623,18.627073,-34.048746,0.000000
